# Cell 1: Setup and Installation

In [ ]:
!pip install librosa scikit-learn matplotlib numpy pandas noisereduce tensorflow shap

# Cell 1.5

In [ ]:
# Cell 1.5: Updated - No unzip needed since files are already extracted
import os
import shutil

# Paths to uploaded files
dataset_path = '/kaggle/input/train-audio'
data_dir = '/kaggle/working/violin-emotion-analysis/data'
csv_path = '/kaggle/input/train-audio/emotion_labels.csv'

# Create the working directory
os.makedirs(data_dir, exist_ok=True)

# Copy all audio files from the dataset to our working directory
audio_source_dir = '/kaggle/input/train-audio/audio'
if os.path.exists(audio_source_dir):
    # Copy all WAV files to our working directory
    for file_name in os.listdir(audio_source_dir):
        if file_name.endswith('.wav'):
            source_path = os.path.join(audio_source_dir, file_name)
            dest_path = os.path.join(data_dir, file_name)
            shutil.copy2(source_path, dest_path)
    
    print(f"Copied audio files to working directory")
else:
    print("Audio folder not found!")

# Check folder structure
print("\nData directory structure:")
for root, dirs, files in os.walk(data_dir):
    level = root.replace(data_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")

# Check if CSV exists
if not os.path.exists(csv_path):
    print("\nWARNING: 'emotion_labels.csv' not found!")
else:
    print(f"\nCSV file found: {csv_path}")
    # Also copy CSV to working directory for consistency
    shutil.copy2(csv_path, '/kaggle/working/violin-emotion-analysis/emotion_labels.csv')
    print("CSV copied to working directory")

print(f"\nTotal audio files ready for processing: {len([f for f in os.listdir(data_dir) if f.endswith('.wav')])}")

# Cell 2: Imports and Updated Utility Functions

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Attention, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import noisereduce as nr
import shap
import joblib
import scipy.signal as signal
from tqdm import tqdm
import warnings
import soundfile as sf
warnings.filterwarnings('ignore')

# Create directories
os.makedirs('/kaggle/working/violin-emotion-analysis/data', exist_ok=True)
os.makedirs('/kaggle/working/violin-emotion-analysis/models', exist_ok=True)

# ===============================================================
# 1️⃣ Feature Extraction Helper (Temporal + Cleaned Audio)
# ===============================================================

def extract_temporal_features(file_path, sr=44100, frame_length=2048, hop_length=512, window_size=3.0, hop_time=1.5):
    """
    Extract richer emotion-aware features from violin recordings.
    Captures pitch, tonality, articulation, dynamics, dissonance, and temporal evolution.
    """
    y, sr = librosa.load(file_path, sr=sr)
    y = nr.reduce_noise(y=y, sr=sr)

    total_duration = librosa.get_duration(y=y, sr=sr)
    step = int(hop_time * sr)
    window = int(window_size * sr)

    feature_sequences = []
    for start in range(0, len(y) - window, step):
        segment = y[start:start + window]
        segment_features = []

        # === Core tone features ===
        mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=13)
        mfccs_delta = librosa.feature.delta(mfccs)
        chroma = librosa.feature.chroma_stft(y=segment, sr=sr)
        chroma_delta = librosa.feature.delta(chroma)
        spectral_contrast = librosa.feature.spectral_contrast(y=segment, sr=sr)

        # === Expression features (temporal dynamics) ===
        rms = librosa.feature.rms(y=segment)  # energy dynamics
        zcr = librosa.feature.zero_crossing_rate(y=segment)  # articulation
        centroid = librosa.feature.spectral_centroid(y=segment, sr=sr)
        bandwidth = librosa.feature.spectral_bandwidth(y=segment, sr=sr)
        flatness = librosa.feature.spectral_flatness(y=segment)
        tempo, _ = librosa.beat.beat_track(y=segment, sr=sr)
        harmony = librosa.feature.tonnetz(y=librosa.effects.harmonic(segment), sr=sr)

        # === Statistical + temporal summarization per segment ===
        # For RMS and ZCR, include their mean, std, and curve (downsampled)
        curve_len = 20  # downsample curve to 20 points
        rms_curve = np.interp(np.linspace(0, rms.shape[1]-1, curve_len), np.arange(rms.shape[1]), rms[0])
        zcr_curve = np.interp(np.linspace(0, zcr.shape[1]-1, curve_len), np.arange(zcr.shape[1]), zcr[0])

        feature_vec = np.hstack([
            np.mean(mfccs, axis=1), np.std(mfccs, axis=1),
            np.mean(mfccs_delta, axis=1), np.std(mfccs_delta, axis=1),
            np.mean(chroma, axis=1), np.mean(chroma_delta, axis=1),
            np.mean(spectral_contrast, axis=1),
            np.mean(centroid), np.std(centroid),
            np.mean(bandwidth),
            np.mean(flatness),
            tempo if not np.isnan(tempo) else 0,
            np.mean(harmony, axis=1),
            np.mean(rms), np.std(rms), rms_curve,  # include RMS curve
            np.mean(zcr), np.std(zcr), zcr_curve   # include ZCR curve
        ])

        feature_sequences.append(feature_vec)

    return np.array(feature_sequences)

# ===============================================================
# 2️⃣ Data Augmentation Helper
# ===============================================================
def augment_audio(y, sr):
    """
    Create slightly altered versions of the same clip to make the model more robust.
    Includes small pitch shifts and speed changes.
    """
    augmented = []

    # Pitch shifts: ±2 semitones
    for n_steps in [-2, 2]:
        augmented.append(librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps))

    # Time stretching using resampling (since librosa.effects.time_stretch expects a spectrogram now)
    for rate in [0.9, 1.1]:
        new_length = int(len(y) / rate)
        augmented.append(librosa.resample(y, orig_sr=sr, target_sr=int(sr * rate))[:new_length])

    return augmented


# ===============================================================
# 3️⃣ Create Temporal Dataset (with Augmented Samples)
# ===============================================================
def create_temporal_dataset(data_dir, emotion_csv_path, augment=True):
    """
    Creates dataset with temporal features and soft emotion labels.
    The CSV must include: filename, emotion, and soft label columns (e.g., happy, sad, angry).
    """
    annotations = pd.read_csv(emotion_csv_path)
    X, y_soft = [], []

    for _, row in tqdm(annotations.iterrows(), total=len(annotations), desc="Creating dataset"):
        file_path = os.path.join(data_dir, row['filename'])
        if not os.path.exists(file_path):
            continue

        # Extract base features
        base_features = extract_temporal_features(file_path)
        X.append(base_features)
        y_soft.append(row[4:].values.astype(float))  # ← soft emotion probabilities
        
        # --- Optional Augmentation ---
        if augment:
            y, sr = librosa.load(file_path, sr=44100)
            y = nr.reduce_noise(y=y, sr=sr)
            y = librosa.util.normalize(y)
            for aug_y in augment_audio(y, sr):
                # Save augmented version's features
                temp_file = "augmented_temp.wav"
                sf.write(temp_file, aug_y, sr)
                aug_features = extract_temporal_features(temp_file)
                X.append(aug_features)
                y_soft.append(row[4:].values.astype(float))
                os.remove(temp_file)

    # Pad sequences for LSTM
    max_len = max(x.shape[0] for x in X)
    num_features = X[0].shape[1]
    X_padded = np.zeros((len(X), max_len, num_features))
    for i, seq in enumerate(X):
        X_padded[i, :seq.shape[0], :] = seq

    return np.array(X_padded), np.array(y_soft)

print("✅ Advanced preprocessing module loaded (with noise reduction, normalization, and augmentation).")

# Cell 3: Data Preparation

In [ ]:
data_dir = '/kaggle/working/violin-emotion-analysis/data'
emotion_csv_path = '/kaggle/working/violin-emotion-analysis/emotion_labels.csv'  # <-- upload your CSV here

if not os.path.exists(emotion_csv_path):
    print("Please upload 'emotion_labels.csv' with soft emotion probabilities.")
else:
    X, y_soft = create_temporal_dataset(data_dir, emotion_csv_path)
    print(f"Loaded {X.shape[0]} samples with {X.shape[2]} features each and {y_soft.shape[1]} soft emotion dimensions.")

# Cell 4: Model Training (LSTM + Random Forest)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Bidirectional, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import joblib
import json
import numpy as np
import os

class HybridEmotionModel:
    def __init__(self, lstm_model, rf_model, input_shape, output_dim):
        self.lstm_model = lstm_model
        self.rf_model = rf_model
        self.input_shape = input_shape
        self.output_dim = output_dim

    @staticmethod
    def build_lstm(input_shape, output_dim):
        inp = Input(shape=input_shape)
        x = Bidirectional(LSTM(128, return_sequences=True))(inp)
        x = LayerNormalization()(x)
        x = Dropout(0.3)(x)
        x = Bidirectional(LSTM(64))(x)
        x = Dropout(0.3)(x)
        lstm_output = Dense(64, activation='relu', name="lstm_output")(x)
        out = Dense(output_dim, activation='softmax')(lstm_output)
        model = Model(inputs=inp, outputs=out)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model

    def predict(self, X):
        lstm_preds = self.lstm_model.predict(X)
        embedding_model = Model(inputs=self.lstm_model.input,
                                outputs=self.lstm_model.get_layer("lstm_output").output)
        embeddings = embedding_model.predict(X)
        rf_preds = self.rf_model.predict(embeddings)
        fused = 0.7 * lstm_preds + 0.3 * rf_preds
        return fused

    def save(self, folder_path='hybrid_model'):
        os.makedirs(folder_path, exist_ok=True)
        # Save LSTM
        self.lstm_model.save(os.path.join(folder_path, 'lstm_model.h5'))
        # Save RF
        joblib.dump(self.rf_model, os.path.join(folder_path, 'rf_model.pkl'))
        # Save metadata
        meta = {
            'input_shape': self.input_shape,
            'output_dim': self.output_dim
        }
        with open(os.path.join(folder_path, 'meta.json'), 'w') as f:
            json.dump(meta, f)
        print(f"✅ Hybrid model saved to '{folder_path}'")

    @classmethod
    def load(cls, folder_path='hybrid_model'):
        lstm_model = tf.keras.models.load_model(os.path.join(folder_path, 'lstm_model.h5'))
        rf_model = joblib.load(os.path.join(folder_path, 'rf_model.pkl'))
        with open(os.path.join(folder_path, 'meta.json'), 'r') as f:
            meta = json.load(f)
        return cls(lstm_model, rf_model, tuple(meta['input_shape']), meta['output_dim'])


# --- Prepare data ---
X_train, X_test, y_train, y_test = train_test_split(X, y_soft, test_size=0.2, random_state=42)
input_shape = (X_train.shape[1], X_train.shape[2])
output_dim = y_train.shape[1]

# --- Train multiple runs and pick best ---
best_r2 = -np.inf
best_hybrid = None

for run_seed in range(3):  # run 3 times, pick best
    print(f"\n🔹 Training run {run_seed+1}")
    np.random.seed(run_seed)
    tf.random.set_seed(run_seed)
    
    lstm_model = HybridEmotionModel.build_lstm(input_shape, output_dim)
    early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    
    lstm_model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=80,
        batch_size=16,
        callbacks=[early_stop],
        verbose=1
    )
    
    embedding_model = Model(inputs=lstm_model.input,
                            outputs=lstm_model.get_layer("lstm_output").output)
    train_embeddings = embedding_model.predict(X_train)
    test_embeddings = embedding_model.predict(X_test)
    
    rf_model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=run_seed)
    rf_model.fit(train_embeddings, y_train)
    
    # Weighted fusion
    lstm_preds = lstm_model.predict(X_test)
    rf_preds = rf_model.predict(test_embeddings)
    fused_preds = 0.7 * lstm_preds + 0.3 * rf_preds
    
    r2 = r2_score(y_test, fused_preds)
    print(f"Run {run_seed+1} Hybrid R² Score: {r2:.4f}")
    
    if r2 > best_r2:
        best_r2 = r2
        best_hybrid = HybridEmotionModel(lstm_model, rf_model, input_shape, output_dim)

# --- Save the best hybrid ---
best_hybrid.save('hybrid_model')
print(f"🏆 Best hybrid model R²: {best_r2:.4f}")

# Cell 5: Explainability and Visualization

In [ ]:
# Cell 5: Enhanced SHAP Analysis with Hybrid Model Embeddings
print("🎵 SHAP Analysis - Feature Importance by Emotion")

import shap
import matplotlib.pyplot as plt
import numpy as np

# --- Define emotion labels ---
emotion_labels = ['Happy', 'Sad', 'Angry']  # Update if needed

# --- Get embeddings for SHAP ---
embedding_model = Model(inputs=best_hybrid.lstm_model.input,
                        outputs=best_hybrid.lstm_model.get_layer("lstm_output").output)

train_embeddings = embedding_model.predict(X_train[:100])  # First 100 training samples
test_embeddings = embedding_model.predict(X_test[:50])     # First 50 test samples

# --- Create SHAP explainer ---
explainer = shap.Explainer(best_hybrid.rf_model, train_embeddings)
shap_values = explainer(test_embeddings)

print("🔍 Analyzing feature importance for each emotion...")

# --- Feature names (one per embedding neuron) ---
num_embeddings = train_embeddings.shape[1]
feature_categories = [f"LSTM_Embedding_{i+1}" for i in range(num_embeddings)]

# --- Analyze SHAP values per emotion ---
for emotion_idx, emotion_name in enumerate(emotion_labels):
    print(f"\n🎻 {emotion_name.upper()} EMOTION:")
    print("="*40)
    
    # Get SHAP values for this specific emotion
    emotion_shap_values = shap_values.values[:, :, emotion_idx]
    
    # Top 10 important embedding neurons
    feature_importance = np.mean(np.abs(emotion_shap_values), axis=0)
    top_feature_indices = np.argsort(feature_importance)[-10:][::-1]
    
    print(f"Top embedding neurons predicting {emotion_name}:")
    for i, idx in enumerate(top_feature_indices):
        importance = feature_importance[idx]
        print(f"  {i+1}. {feature_categories[idx]}: {importance:.4f}")

# --- SHAP summary plot per emotion ---
for emotion_idx, emotion_name in enumerate(emotion_labels):
    print(f"\n📈 Generating SHAP summary plot for {emotion_name} embeddings...")
    shap.summary_plot(
        shap_values.values[:, :, emotion_idx],   # Only this emotion
        test_embeddings,
        feature_names=feature_categories,
        show=False
    )
    plt.title(f"SHAP Feature Importance - {emotion_name}")
    plt.tight_layout()
    plt.show()

# --- Optional: Detailed explanation for first test sample ---
print("\n🔬 Detailed prediction breakdown for first test sample:")
sample_idx = 0
shap.force_plot(
    explainer.expected_value[0], 
    shap_values.values[sample_idx, :, 0], 
    test_embeddings[sample_idx, :],
    feature_names=feature_categories,
    matplotlib=True,
    show=False
)
plt.title(f"Prediction Breakdown: {emotion_labels[np.argmax(y_test[sample_idx])]} → {emotion_labels[np.argmax(best_hybrid.predict(X_test[sample_idx:sample_idx+1]))]}")
plt.tight_layout()
plt.show()
